In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!apt install git-lfs



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
%cd /content/drive/MyDrive/
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/OpenGVLab/InternVL2_5-1B


/content/drive/MyDrive
Cloning into 'InternVL2_5-1B'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 74 (delta 27), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (74/74), 1.91 MiB | 1.39 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/InternVL2_5-1B/.git/hooks/post-checkout': Permission denied


In [ ]:
!ls -lh /content/drive/MyDrive/InternVL2_5-1B


total 5.0M
-rw------- 1 root root  790 Feb  8 11:48 added_tokens.json
-rw------- 1 root root 3.7K Feb  8 11:48 config.json
-rw------- 1 root root 5.5K Feb  8 11:48 configuration_intern_vit.py
-rw------- 1 root root 3.8K Feb  8 11:48 configuration_internvl_chat.py
-rw------- 1 root root   41 Feb  8 11:48 configuration.json
-rw------- 1 root root  15K Feb  8 11:48 conversation.py
drwx------ 2 root root 4.0K Feb  8 11:48 examples
-rw------- 1 root root  129 Feb  8 11:48 generation_config.json
-rw------- 1 root root 1.6M Feb  8 11:48 merges.txt
-rw------- 1 root root  18K Feb  8 11:48 modeling_intern_vit.py
-rw------- 1 root root  16K Feb  8 11:48 modeling_internvl_chat.py
-rw------- 1 root root  135 Feb  8 11:48 model.safetensors
-rw------- 1 root root  287 Feb  8 11:48 preprocessor_config.json
-rw------- 1 root root  34K Feb  8 11:48 README.md
drwx------ 3 root root 4.0K Feb  8 11:48 runs
-rw------- 1 root root  744 Feb  8 11:48 special_tokens_map.json
-rw------- 1 root root 8.9K Feb  8 

In [ ]:
import torch
from transformers import AutoModel, AutoProcessor
from PIL import Image
import torchvision.transforms as transforms

# Define model path
path = "OpenGVLab/InternVL2_5-1B"
local_dir = "/content/drive/MyDrive/InternVL2_5-1B"

# Ensure device compatibility
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32  # Ensure model and input types match

# Load the model
model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

# Load processor (acts as tokenizer)
processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

# Function to preprocess image (Fix: Convert to torch.bfloat16 if using GPU)
def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension & move to device
    return image.to(torch_dtype)  # Ensure correct dtype for model

# Load and preprocess the image
image_path = "/content/drive/MyDrive/P_DOC/HealthID.jpeg"  # Replace with your actual image path
image = preprocess_image(image_path)

# Define text input
text = "Describe the image."

# Fix: Use processor directly for text
inputs = processor(text=[text], return_tensors="pt").to(device)

# Fix: Ensure inputs are converted to model dtype
inputs["pixel_values"] = image

# Run model inference
with torch.no_grad():
    response = model.chat(processor, inputs["pixel_values"], text, generation_config={"max_new_tokens": 512})

print("Assistant:", response)


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: The image shows a scanned copy of an Aadhaar card from the National Aadhaar Authority of India. The card includes the following details:

- **Name:** Mahidumitha
- **Aadhaar Number:** 91-226-2277-4311
- **Full Name:** Mahidumitha
- **Gender:** Female
- **Date of Birth:** 05-12-2004
- **Mobile Number:** 970598585
- **QR Code:** Present on the card

The card also includes a description of Aadhaar and its benefits, as well as a QR code for digital health services.


In [ ]:
import torch
import time
from transformers import AutoModel, AutoProcessor
from PIL import Image
import torchvision.transforms as transforms

path = "OpenGVLab/InternVL2_5-1B"
local_dir = "/content/drive/MyDrive/InternVL2_5-1B"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device).to(torch_dtype)

image_path = "/content/sample.jpeg"
image = preprocess_image(image_path)

text = "You are a document ID API that extracts and sends details from driving licenses in JSON format . Extract the following details: Document Type, Issuing Authority, Name, Date of Birth, Gender, Address or Location (Location should be separate), Document Number, Expiry Date, Nationality, Other Identifiers.If the image is upside down,turned left or right ,  fix it and get the details. If the image is missing details please set the value as empty string like this  \"\" , If the image is not looks like ID means send an empty JSON. Only reply in JSON"

inputs = processor(text=[text], return_tensors="pt").to(device)
inputs["pixel_values"] = image

start_time = time.time()

with torch.no_grad():
    response = model.chat(processor, inputs["pixel_values"], text, generation_config={"max_new_tokens": 512})

end_time = time.time()
inference_time = end_time - start_time

print("Assistant:", response)
print(f"Inference Time: {inference_time:.4f} seconds")


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: ```json
{
  "Document Type": "Driver's License",
  "Issuing Authority": "Western Australia",
  "Name": "MELSAUROU ROBERT PAUL",
  "Date of Birth": "18 Apr 2027",
  "Gender": "Male",
  "Address": "ELLENROOK, WA 6060",
  "Document Number": "LICENCE1024",
  "Expiry Date": "07 Oct 1983",
  "Nationality": "",
  "Other Identifiers": ""
}
```
Inference Time: 9.0954 seconds


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch
import time
from transformers import AutoModel, AutoProcessor
from PIL import Image, ImageFile
import torchvision.transforms as transforms

ImageFile.LOAD_TRUNCATED_IMAGES = True

path = "OpenGVLab/InternVL2_5-1B"
local_dir = "/content/drive/MyDrive/InternVL2_5-1B"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    try:
        image = Image.open(image_path).convert("RGB")
    except OSError:
        print(f"Error loading image: {image_path}. Check if the file is corrupted.")
        return None
    return transform(image).unsqueeze(0).to(device).to(torch_dtype)

image_path = "/content/hariprem.jpg"
image = preprocess_image(image_path)

if image is None:
    exit()

text = "You are a document ID API that extracts and sends details from driving licenses in JSON format . Extract the following details: Document Type, Issuing Authority, Name, Date of Birth, Gender, Address or Location (Location should be separate), Document Number, Expiry Date, Nationality, Other Identifiers.If the image is upside down,turned left or right ,  fix it and get the details. If the image is missing details please set the value as empty string like this  \"\" , If the image is not looks like ID means send an empty JSON. Only reply in JSON"

inputs = processor(text=[text], return_tensors="pt").to(device)
inputs["pixel_values"] = image

start_time = time.time()

with torch.no_grad():
    response = model.chat(processor, inputs["pixel_values"], text, generation_config={"max_new_tokens": 512})

end_time = time.time()
inference_time = end_time - start_time

print("Assistant:", response)
print(f"Inference Time: {inference_time:.4f} seconds")


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: ```json
{
  "Document Type": "Driving License",
  "Issuing Authority": "TNI Malang",
  "Name": "HARI PERC",
  "Date of Birth": "27-05-1992",
  "Gender": "",
  "Address or Location": "",
  "Document Number": "TAS123456789",
  "Expiry Date": "",
  "Nationality": "",
  "Other Identifiers": ""
}
```
Inference Time: 4.3651 seconds


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch
import time
from transformers import AutoModel, AutoProcessor
from PIL import Image, ImageFile
import torchvision.transforms as transforms

ImageFile.LOAD_TRUNCATED_IMAGES = True

path = "OpenGVLab/InternVL2_5-1B"
local_dir = "/content/drive/MyDrive/InternVL2_5-1B"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    try:
        image = Image.open(image_path).convert("RGB")
    except OSError:
        print(f"Error loading image: {image_path}. Check if the file is corrupted.")
        return None
    return transform(image).unsqueeze(0).to(device).to(torch_dtype)

image_path = "/content/arif.JPG"
image = preprocess_image(image_path)

if image is None:
    exit()

text = "You are a document ID API that extracts and sends details from driving licenses in JSON format . Extract the following details: Document Type, Issuing Authority, Name, Date of Birth, Gender, Address or Location (Location should be separate), Document Number, Expiry Date, Nationality, Other Identifiers.If the image is upside down,turned left or right ,  fix it and get the details. If the image is missing details please set the value as empty string like this  \"\" , If the image is not looks like ID means send an empty JSON. Only reply in JSON"

inputs = processor(text=[text], return_tensors="pt").to(device)
inputs["pixel_values"] = image

start_time = time.time()

with torch.no_grad():
    response = model.chat(processor, inputs["pixel_values"], text, generation_config={"max_new_tokens": 512})

end_time = time.time()
inference_time = end_time - start_time

print("Assistant:", response)
print(f"Inference Time: {inference_time:.4f} seconds")


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: ```json
{
  "Document Type": "Driving License (Tamil Nadu)",
  "Issuing Authority": "TNSS",
  "Name": "Abdullah Aboodherer",
  "Date of Birth": "19-11-1980",
  "Gender": "",
  "Address or Location": "",
  "Document Number": "TNSS20200000000000004",
  "Expiry Date": "",
  "Nationality": "",
  "Other Identifiers": ""
}
```
Inference Time: 9.5662 seconds


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch
import time
from transformers import AutoModel, AutoProcessor
from PIL import Image, ImageFile
import torchvision.transforms as transforms

ImageFile.LOAD_TRUNCATED_IMAGES = True

path = "OpenGVLab/InternVL2_5-1B"
local_dir = "/content/drive/MyDrive/InternVL2_5-1B"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

model = AutoModel.from_pretrained(
    path,
    cache_dir=local_dir,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

processor = AutoProcessor.from_pretrained(path, cache_dir=local_dir, trust_remote_code=True)

def preprocess_image(image_path, image_size=448):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    try:
        image = Image.open(image_path).convert("RGB")
    except OSError:
        print(f"Error loading image: {image_path}. Check if the file is corrupted.")
        return None
    return transform(image).unsqueeze(0).to(device).to(torch_dtype)

image_path = "/content/mukundan.jpeg"
image = preprocess_image(image_path)

if image is None:
    exit()

text = "You are a document ID API that extracts and sends details from driving licenses in JSON format . Extract the following details: Document Type, Issuing Authority, Name, Date of Birth, Gender, Address or Location (Location should be separate), Document Number, Expiry Date, Nationality, Other Identifiers.If the image is upside down,turned left or right ,  fix it and get the details. If the image is missing details please set the value as empty string like this  \"\" , If the image is not looks like ID means send an empty JSON. Only reply in JSON"

inputs = processor(text=[text], return_tensors="pt").to(device)
inputs["pixel_values"] = image

start_time = time.time()

with torch.no_grad():
    response = model.chat(processor, inputs["pixel_values"], text, generation_config={"max_new_tokens": 512})

end_time = time.time()
inference_time = end_time - start_time

print("Assistant:", response)
print(f"Inference Time: {inference_time:.4f} seconds")


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Assistant: ```json
{
  "Document Type": "Union of India Driving Licence (Tamil Nadu)",
  "Issuing Authority": "Union of India",
  "Name": "MUKUNTHAN KALAISELVAN",
  "Date of Birth": "02-05-2001",
  "Gender": "O+",
  "Address or Location": "",
  "Document Number": "TN85 20200000198",
  "Expiry Date": "01-05-2041",
  "Nationality": "",
  "Other Identifiers": ""
}
```
Inference Time: 8.3844 seconds
